In [0]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")
train_df

In [3]:
print(train_df.shape)
print(test_df.shape)

(891, 12)
(418, 11)


In [14]:
# isna, sum: train_df.isna().sum()
s = train_df.isna().sum()
s[s > 0]

Cabin       687
Embarked      2
dtype: int64

In [15]:
# isna, sum: train_df.isna().sum()
s = test_df.isna().sum()
s[s > 0]

Cabin    327
dtype: int64

In [13]:
# 數值類型缺失直: 補中位數
# medain(), mean()
# Pclass: 其實是類別, 不該填中位數
med = train_df.median().drop(["Pclass"])
train_df = train_df.fillna(med)
# test要不要重算一次med? 不能重算!!
test_df = test_df.fillna(med)
# isna, sum: train_df.isna().sum()
s = test_df.isna().sum()
s[s > 0]

Cabin    327
dtype: int64

In [19]:
# value_counts/idxmax
# 類別型態補缺失職: 捕最常出現
most = train_df["Embarked"].value_counts().idxmax()
train_df["Embarked"] = train_df["Embarked"].fillna(most)
s = train_df.isna().sum()
s[s > 0]

Cabin    687
dtype: int64